# Modelo A para clasificación de imagenes

#### Jeffrey Daniel Leiva Cascante 2021016720
#### Richard Osvaldo León Chinchilla 2019003759

## Inicialización

In [10]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resnet es de 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.RandomRotation(20),  # Rotación aleatoria
    transforms.RandomHorizontalFlip(),  # Inversión horizontal aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Variación de brillo y contraste
])

# Cargamos el dataset de entrenamiento para cada dataset
train_dataset = datasets.ImageFolder(root='./Covid19-dataset/train', transform=transform)
train_dataset_bilateral = datasets.ImageFolder(root='./Covid19-dataset-bilateral/train', transform=transform)
train_dataset_canny = datasets.ImageFolder(root='./Covid19-dataset-canny/train', transform=transform)

val_percent = 0.15 # Porcentaje de imágenes que se usarán para validación

# Calculamos el tamaño de los conjuntos de entrenamiento y validación
val_size = int(val_percent * len(train_dataset))
train_size = len(train_dataset) - val_size

val_size_bilateral = int(val_percent * len(train_dataset_bilateral))
train_size_bilateral = len(train_dataset_bilateral) - val_size_bilateral

# Se divide el dataset en dos
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"Train dataset size: {len(train_dataset)}")
print(f"train_dataset_bilateral: {len(train_dataset_bilateral)}")

# Creamos los dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

Train dataset size: 211
train_dataset_bilateral: 248


### Weights and Biases

In [11]:
import wandb
wandb.login(
    key="52cc20b894f4ec68c5e30b411f8f55148e7e54ec"
)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## Carga de Resnet-50 preentrenado

In [12]:
import torch.nn as nn
import torchvision.models as models

# Cargamos el modelo preentrenado
resnet50  = models.resnet50(pretrained=True)

# Se modifica la capa final para adaptarla a 3 clases
num_classes = 3
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)

# Se congela el modelo para que no se actualicen los pesos de las capas preentrenadas
for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.fc.parameters():
    param.requires_grad = True

# Se mueve el modelo a la GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet50 = resnet50.to(device)

# Solo los parametros de la capa final se entrenan
optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=0.001)

# Se define la función de pérdida
criterion = nn.CrossEntropyLoss()


c:\Users\richa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\richa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Entrenamiento

In [13]:
wandb.init(project='Model-A')

wandb.config={
    'batch_size': 32,
    'learning_rate': 0.001,
    'epochs': 10,
    'optimizer': 'Adam',
}
wandb.watch(resnet50, log='all', log_freq=10)


def train_model(train_loader,val_loader,n_epochs):
    for epoch in range(n_epochs):
        resnet50.train()
        train_loss = 0.0 
        # Entrenamiento
        for input,labels in train_loader: # Se recorren los datos de entrenamiento
            inputs, labels = input.to(device), labels.to(device)
            optimizer.zero_grad() # Se reinician los gradientes
            outputs = resnet50(inputs) # Se obtiene la salida del modelo
            loss = criterion(outputs, labels) # Se calcula la pérdida
 
            loss.backward() # Se calculan los gradientes
            optimizer.step() # Se actualizan los pesos

            train_loss += loss.item() * inputs.size(0)
            _,predicted = torch.max(outputs, 1)
            corrects = torch.sum(predicted == labels.data)

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = corrects.double() / len(train_loader.dataset)

        #Validación
        resnet50.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad(): # No se calculan los gradientes en la validación
            for input,labels in val_loader:
                inputs, labels = input.to(device), labels.to(device)
                outputs = resnet50(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                _,predicted = torch.max(outputs, 1)
                val_corrects += torch.sum(predicted == labels.data)

            val_loss = val_loss/len(val_loader.dataset)
            val_acc = val_corrects.double() / len(val_loader.dataset)

            wandb.log({
                "Epoch": epoch,
                "Train Loss": train_loss,
                "Train Acc": train_acc,
                "Val Loss": val_loss,
                "Val Acc": val_acc,
            })
            print('Epoch: {} Train Loss: {:.4f} Train Acc: {:.4f} Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch, train_loss, train_acc, val_loss, val_acc))




### Dataset Crudo

In [14]:
train_model(train_loader, val_loader, 10)

KeyboardInterrupt: 

In [ ]:
wandb.finish()